Graph convolutional network for fMRI analysis based on connectivity neighborhood:
The paper outlines a methodology for converting functional connectivity (FC) matrices from fMRI data into graph structures using a k-nearest neighbors (k-NN) approach. The k-NN graph is built by connecting each node (ROI) to its k nearest neighbors based on the strength of connectivity, which is represented by the FC matrix values.

In [1]:
import nibabel as nib
import numpy as np
import networkx as nx
import os
import math
import pandas as pd
from tqdm.notebook import tqdm  # Import tqdm for notebooks
import matplotlib.pyplot as plt


In [2]:
def load_fc_matrix(file_path):
    """ Load functional connectivity matrix from a .pconn.nii file. """
    img = nib.load(file_path)
    fc_matrix = img.get_fdata()
    return fc_matrix

def create_knn_graph(fc_matrix, k=5):
    """ Create a graph from a functional connectivity matrix using k-nearest neighbors based on absolute values. """
    n = fc_matrix.shape[0]  # Number of nodes
    G = nx.Graph()
    for i in range(n):
        G.add_node(i)
    
    # For each node, add edges to the k-nearest neighbors based on absolute values of connectivity strengths
    for i in range(n):
        # Sort indices based on the absolute values, get the k highest values indices for each row
        indices = np.argsort(np.abs(fc_matrix[i]))[-k:]
        for j in indices:
            if i != j:  # Ensure no self-loops
                G.add_edge(i, j, weight=fc_matrix[i][j])
    
    return G

In [4]:
# Directory containing the pconn files
directory = "C:/Users/manfr/OneDrive/Escritorio/Manfred/EMAI/SecondSemester_Slovenia/Project/BSNIP_neural/BSNIP/pconn"
pconn_files = [f for f in os.listdir(directory) if f.endswith('.pconn.nii')]

# Prepare a list to store the results
results = []

for file_name in tqdm(pconn_files, desc="Processing .pconn.nii files"):
    fc_file_path = os.path.join(directory, file_name)
    fc_matrix = load_fc_matrix(fc_file_path)
    graph = create_knn_graph(fc_matrix, k=5)

    degrees = [deg for _, deg in graph.degree()]
    n = graph.number_of_nodes()
    if n > 1:  # To avoid division by zero in calculations
        average_degree = sum(degrees) / n
        theoretical_avg_c = average_degree / (n - 1)
        theoretical_avg_d = math.log(n) / math.log(average_degree) if average_degree > 1 else 0

        # Calculate clustering and path length on the largest connected component
        largest_cc = max(nx.connected_components(graph), key=len)
        subgraph = graph.subgraph(largest_cc)
        size_of_largest_cc = len(largest_cc)
        avg_clustering = nx.average_clustering(graph)
        avg_path_length = nx.average_shortest_path_length(subgraph) if len(largest_cc) > 1 else 0

        row = [
            file_name, n, average_degree, theoretical_avg_c, avg_clustering,
            theoretical_avg_d, avg_path_length, math.log(n), math.log(math.log(n)),
            size_of_largest_cc
        ]
    else:
        row = [file_name, n, 0, 0, 0, 0, 0, 0, 0, 0]

    results.append(row)
# Create a DataFrame
df = pd.DataFrame(results, columns=[
    'File Name', 'Number of Nodes', 'Average Degree', 'Theoretical Avg Clustering',
    'Average Clustering', 'Theoretical Avg Path Length', 'Average Path Length',
    'Log of Nodes', 'Log Log of Nodes', 'Size of Largest CC'
])

# Save to CSV
df.to_csv('graph_statistics.csv', index=False)
print("Data saved to 'graph_statistics.csv'.")


Processing .pconn.nii files:   0%|          | 0/638 [00:00<?, ?it/s]

Data saved to 'graph_statistics.csv'.


In [8]:
import subprocess
import os

def orca(G, exe_folder=".", output_folder="."):
    if "orca.exe" not in os.listdir(exe_folder):
        raise Exception(exe_folder + " doesn't contain orca.exe")

    input_filename = os.path.join(exe_folder, G.name + ".in")
    output_filename = os.path.join(output_folder, G.name + ".orca")

    # Write the input file
    with open(input_filename, 'w') as file:
        file.write(str(G.number_of_nodes()) + " " + str(G.number_of_edges()) + "\n")
        for i, j in G.edges():
            file.write(f"{i} {j}\n")

    # Construct the command
    command = [os.path.join(exe_folder, 'orca.exe'), "node", "4", input_filename, output_filename]
    print("Running command:", " ".join(command))  # Print the command for debugging

    # Run the command
    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("STDOUT:", result.stdout)
        print("STDERR:", result.stderr)
    except subprocess.CalledProcessError as e:
        print("ERROR:", e.stderr)
        print("STDOUT:", e.stdout)
        return None
    except Exception as e:
        print("Failed to execute command:", e)
        return None

    # Check if the output file was created and read it
    if not os.path.exists(output_filename) or os.stat(output_filename).st_size == 0:
        print("Output file not created or is empty.")
        return None

    # Read the results
    orbits = []
    with open(output_filename, 'r') as file:
        for line in file:
            orbits.append([int(k) for k in line.split()])

    # Clean up files
    os.remove(input_filename)
    # Optionally remove the output file after processing, if desired
    # os.remove(output_filename)

    return orbits


In [9]:
# Example usage of the modified function
orca_folder = 'C:/Users/manfr/OneDrive/Escritorio/Manfred/EMAI/SecondSemester_Slovenia/Project/classifying_psychiatric_disorders/src/orca'
output_folder = 'C:/Users/manfr/OneDrive/Escritorio/Manfred/EMAI/SecondSemester_Slovenia/Project/classifying_psychiatric_disorders/src/results_orca'
directory = "C:/Users/manfr/OneDrive/Escritorio/Manfred/EMAI/SecondSemester_Slovenia/Project/BSNIP_neural/BSNIP/pconn"
pconn_files = [f for f in os.listdir(directory) if f.endswith('.pconn.nii')]

fc_file_path = os.path.join(directory, pconn_files[0])
fc_matrix = load_fc_matrix(fc_file_path)
graph = create_knn_graph(fc_matrix, k=5)
graph.name = "ExampleGraph"  # Ensure the graph has a name attribute
orbits1 = orca(graph, exe_folder=orca_folder, output_folder=output_folder)

Running command: C:/Users/manfr/OneDrive/Escritorio/Manfred/EMAI/SecondSemester_Slovenia/Project/classifying_psychiatric_disorders/src/orca\orca.exe node 4 C:/Users/manfr/OneDrive/Escritorio/Manfred/EMAI/SecondSemester_Slovenia/Project/classifying_psychiatric_disorders/src/orca\ExampleGraph.in C:/Users/manfr/OneDrive/Escritorio/Manfred/EMAI/SecondSemester_Slovenia/Project/classifying_psychiatric_disorders/src/results_orca\ExampleGraph.orca
ERROR: 
STDOUT: 


In [10]:
print(orbits1)

None
